# MUVERA (ColBERT Multi-Vector) 評価

WeaviateのMUVERAエンコーディングを使用してColBERTマルチベクトル検索の性能を評価します。

- **データセット**: JQaRA (Japanese Question-Answer Retrieval Assessment)
- **エンベディング**: JaColBERT (bclavie/JaColBERT)
- **評価方式**: 各クエリに対して100件の候補文書をリランキング

## 1. Weaviate接続とコレクション名定義

In [ ]:
import weaviate

client = weaviate.connect_to_local()
MUVERA_CLASS_NAME = "jqara_muvera"

## 2. MUVERAコレクション作成

MUVERAエンコーディングを使用したマルチベクトルコレクションを作成します。
- `q_id`: クエリIDでフィルタリングするためのプロパティ（インデックス有効）
- `multi_vector`: ColBERTのトークンレベルエンベディングを格納

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

if client.collections.exists(MUVERA_CLASS_NAME):
    client.collections.delete(MUVERA_CLASS_NAME)

client.collections.create(
    name=MUVERA_CLASS_NAME,
    vector_config=[
        Configure.MultiVectors.self_provided(
            name="multi_vector",
            encoding=Configure.VectorIndex.MultiVector.Encoding.muvera(),
        ),
    ],
    properties=[
        Property(name="d_id", data_type=DataType.TEXT),
        Property(name="q_id", data_type=DataType.TEXT, index_filterable=True),
        Property(name="title", data_type=DataType.TEXT),
        Property(name="text", data_type=DataType.TEXT),
    ],
)

muvera_col = client.collections.get(MUVERA_CLASS_NAME)

## 3. JQaRAデータセット読み込み

JQaRAテストデータを読み込みます。各レコードには以下が含まれます：
- `id`: ドキュメントID
- `q_id`: クエリID（同一クエリに対する複数の候補文書をグループ化）
- `label`: 正解ラベル（1=関連, 0=非関連）
- `text`, `title`: 文書内容

In [ ]:
import json
import os
from pathlib import Path

dataset = []
# notebookディレクトリからの相対パス
project_root = Path(os.getcwd()).parent
file_path = project_root / "data" / "jqara_test.jsonl"

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        if not line.strip():
            continue
        obj = json.loads(line)        
        dataset.append(obj)

print(dataset[0])

## 4. ColBERTエンベディング読み込み

`encoder.py`で事前生成したColBERTマルチベクトルエンベディングを読み込みます。
各ドキュメントは `[n_tokens, dim]` 形状のテンソルを持ちます。

In [ ]:
import glob
import torch

batch_paths = sorted(glob.glob(str(project_root / "outputs" / "jacolbert" / "batch_*.pt")))

ids = []
embeddings = []  # List[Tensor[n_tokens, dim]]

for p in batch_paths:
    payload = torch.load(p, map_location="cpu")
    ids.extend([str(x) for x in payload.get("ids", [])])
    for e in payload.get("embeddings", []):
        if isinstance(e, torch.Tensor):
            embeddings.append(e.detach().to("cpu").contiguous())
        else:
            embeddings.append(torch.as_tensor(e, dtype=torch.float32))

assert len(ids) == len(embeddings)

print("batches:", len(batch_paths), "docs:", len(embeddings))

## 5. Weaviateへデータアップロード

ドキュメントとColBERTエンベディングをMUVERAコレクションにバッチアップロードします。

In [ ]:
from tqdm import tqdm

N = 166700
subset = dataset[:N]
emb_subset = embeddings[:N]

total = len(subset)

with muvera_col.batch.fixed_size(batch_size=16) as batch, tqdm(total=total, desc="📤 Uploading to Weaviate") as pbar:
    for obj, emb in zip(subset, emb_subset):
        batch.add_object(
            properties={
                "d_id": obj["id"],
                "q_id": obj["q_id"],
                "text": obj["text"],
                "title": obj["title"],
            },
            vector={"multi_vector": emb.tolist()},
        )
        pbar.update(1)

if muvera_col.batch.failed_objects:
    print(f"\n❌ Number of failed imports: {len(muvera_col.batch.failed_objects)}")
    print(f"First failed object: {muvera_col.batch.failed_objects[0]}")
else:
    print("\n✅ All objects uploaded successfully!")

count = muvera_col.aggregate.over_all(total_count=True).total_count
print(f"🧮 現在の登録件数: {count}")

## 6. ColBERTチェックポイント読み込み

クエリエンコード用にJaColBERTモデルを読み込みます。

In [ ]:
from colbert.infra import ColBERTConfig
from colbert.modeling.checkpoint import Checkpoint

ckpt = Checkpoint("bclavie/JaColBERT", colbert_config=ColBERTConfig())

## 7. 評価用データ構造作成

ranx評価用のqrels（正解ラベル）とqid_to_query（クエリマッピング）を作成します。

In [46]:
from typing import Dict, List, Tuple, Iterable


qid_to_query: Dict[str, str] = {}
qrels: Dict[str, Dict[str, int]] = {}

cnt = 0
for rec in subset:
    cnt += 1
    qid   = rec["q_id"]
    q = rec.get("question")
    docid = rec.get("id")
    rel   = int(rec.get("label", 0))

    if qid not in qid_to_query:
        qid_to_query[qid] = q
    qrels.setdefault(qid, {})[docid] = rel

print(f"Loaded {cnt} records")
print(f"Unique queries: {len(qid_to_query)}")

Loaded 166700 records
Unique queries: 1667


## 8. 非同期検索ヘルパー関数

全クエリを並列実行してrun_dict（検索結果）とレイテンシを収集する関数を定義します。

In [ ]:
from typing import Callable, Awaitable, List, Tuple, Dict
import asyncio

async def build_run_dict_async_with_qid(
        search_fn: Callable[[str, str, int], Awaitable[Tuple[List[Tuple[str, float]], float]]],
        qid_to_query: Dict[str, str],
        topk: int = 100
    ) -> Tuple[Dict[str, Dict[str, float]], List[float]]:
    """
    非同期検索関数を使って全クエリを実行し、run_dictとlatenciesを返す。
    q_idでフィルタリングするバージョン。
    
    Args:
        search_fn: (q, q_id, topk) -> ([(doc_id, score), ...], elapsed)
        qid_to_query: {qid: query_text} の辞書
        topk: 各クエリで取得する件数
        
    Returns:
        run_dict: {qid: {doc_id: score, ...}, ...}
        latencies: 各クエリのレイテンシリスト
    """
    CONCURRENCY = 10
    sem = asyncio.Semaphore(CONCURRENCY)    
    run_dict: Dict[str, Dict[str, float]] = {}
    latencies: List[float] = []

    async def _one(qid: str, q: str):
        async with sem:
            ranked, elapsed = await search_fn(q, qid, topk)
            return qid, ranked, elapsed

    tasks = [_one(qid, q) for qid, q in qid_to_query.items()]
    
    for coro in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Searching"):
        qid, ranked, elapsed = await coro
        run_dict[qid] = {doc_id: score for doc_id, score in ranked}
        latencies.append(elapsed)
    
    return run_dict, latencies

## 9. MUVERA検索関数

q_idでフィルタリングしてColBERTマルチベクトル検索を実行する関数を定義します。
- クエリをColBERTでエンコード
- 該当q_idの候補文書群内でnear_vector検索
- distanceを反転してスコア化（ranxは大きいスコアを上位とするため）

In [57]:
from typing import List, Tuple
from weaviate.classes.query import MetadataQuery, Filter
import time

async def async_muvera_search(async_col, q: str, q_id: str, topk: int = 100) -> Tuple[List[Tuple[str, float]], float]:
    """
    ColBERT (Multi-vector) を用いた非同期検索関数
    q_idでフィルタリングして、該当クエリの候補文書群の中からランキング
    
    Args:
        async_col: 非同期Weaviateコレクション
        q: 検索クエリ文字列
        q_id: フィルタ用のクエリID
        topk: 取得する件数
        
    Returns:
        ([(doc_id, score), ...], elapsed) のタプル
        scoreは大きいほど良い（distanceを反転）
    """
    # クエリのベクトル化 (ColBERT)
    q_vecs_tensor = ckpt.queryFromText([q], bsize=1)[0]
    q_vecs = q_vecs_tensor.tolist()

    # 非同期クエリの実行（q_idでフィルタリング）
    start = time.perf_counter()

    response = await async_col.query.near_vector(
        near_vector=q_vecs,
        target_vector="multi_vector",
        limit=topk,
        filters=Filter.by_property("q_id").equal(q_id),
        return_metadata=MetadataQuery(distance=True)
    )

    elapsed = time.perf_counter() - start

    # 結果の整形: (doc_id, score) のリスト
    # ranxは大きいスコアを上位とするため、distanceを反転（負にする）
    results: List[Tuple[str, float]] = []
    for obj in response.objects:
        doc_id = obj.properties.get("d_id", "")
        score = -obj.metadata.distance  # 距離を反転！
        results.append((doc_id, score))

    return results, elapsed


def create_muvera_search_fn(async_col):
    """async_colをバインドした検索関数を返すファクトリ（q_id対応版）"""
    async def search_fn(q: str, q_id: str, topk: int = 100) -> Tuple[List[Tuple[str, float]], float]:
        return await async_muvera_search(async_col, q, q_id, topk)
    return search_fn

## 10. 全クエリで検索実行

1667クエリすべてに対してMUVERA検索を実行し、結果とレイテンシを収集します。

In [59]:
from tqdm import tqdm

# build_run_dict_async_with_qid を使って全クエリを検索（q_idフィルタリング版）

muvera_search_run_dict = {}
muvera_search_latencies = []

async with weaviate.use_async_with_local() as async_client:
    async_col = async_client.collections.get(MUVERA_CLASS_NAME)
    
    # async_col をバインドした検索関数を作成
    search_fn = create_muvera_search_fn(async_col)
    
    # 全クエリを実行（q_idでフィルタリング）
    muvera_search_run_dict, muvera_search_latencies = await build_run_dict_async_with_qid(
        search_fn, 
        qid_to_query,
        topk=100  # 各q_idに対応する全ドキュメントを取得
    )

print(f"検索完了: {len(muvera_search_run_dict)} クエリ")
print(f"平均レイテンシ: {sum(muvera_search_latencies) / len(muvera_search_latencies):.4f} 秒")

Searching: 100%|██████████| 1667/1667 [00:42<00:00, 39.16it/s]

検索完了: 1667 クエリ
平均レイテンシ: 0.1789 秒


## 11. 評価関数定義

ranxを使用してNDCG, MAP, MRR, Precision, Recallを計算する関数を定義します。

In [63]:
from ranx import Qrels, Run, evaluate

def evaluate_run_dict(run_dict: Dict[str, Dict[str, float]], qrels: Dict[str, Dict[str, int]]):
    qrels_ranx = Qrels(qrels)
    run_ranx   = Run(run_dict)

    metrics = [
        "ndcg@1", "ndcg@3", "ndcg@5", "ndcg@10",
        "map", "mrr", "mrr@10", "precision@10", "recall@10", "recall@100",
    ]

    scores = evaluate(qrels_ranx, run_ranx, metrics=metrics)
    for m in metrics:
        print(f"{m:>12}: {scores[m]:.4f}")

## 12. レイテンシ統計関数

In [53]:
import numpy as np

def print_latency_stats(latencies: List[float]):
    l = np.array(latencies)

    print(f"Count    : {len(l)}")
    print(f"Mean     : {l.mean():.4f} sec")
    print(f"Median   : {np.median(l):.4f} sec")
    print(f"p90      : {np.percentile(l,90):.4f} sec")
    print(f"p95      : {np.percentile(l,95):.4f} sec")
    print(f"p99      : {np.percentile(l,99):.4f} sec")
    print(f"Min/Max  : {l.min():.4f} / {l.max():.4f} sec")

## 13. 評価結果

MUVERA検索の精度とレイテンシを表示します。

In [ ]:
evaluate_run_dict(muvera_search_run_dict, qrels)
print_latency_stats(muvera_search_latencies)

      ndcg@1: 0.7169
      ndcg@3: 0.5892
      ndcg@5: 0.5445
     ndcg@10: 0.5144
         map: 0.4426
         mrr: 0.7943
      mrr@10: 0.7910
precision@10: 0.3332
   recall@10: 0.4133
  recall@100: 1.0000
Count    : 1667
Mean     : 0.1789 sec
Median   : 0.1759 sec
p90      : 0.2274 sec
p95      : 0.2413 sec
p99      : 0.3431 sec
Min/Max  : 0.0272 / 0.5739 sec
